### Extracting The Datasets

In [ ]:
import io
import time
from datetime import timedelta
import gensim

if __name__ == '__main__':
    start_time = time.time()
    print('Streaming wiki...')
    id_wiki = gensim.corpora.WikiCorpus(
        './datasets/idwiki-latest-pages-articles.xml.bz2',
        dictionary={}, lower=True
    )
    
    article_count = 0
    with io.open('./datasets/idwiki_new_lower.txt', 'w', encoding='utf-8') as wiki_txt:
        for text in id_wiki.get_texts():

            wiki_txt.write(" ".join(text) + '\n')
            article_count += 1

            if article_count % 10000 == 0:
                print('{} articles processed'.format(article_count))
        print('total: {} articles'.format(article_count))

    finish_time = time.time()
    print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

### Training Word2Vec

In [ ]:
import time
import multiprocessing
from datetime import timedelta
from gensim.models import word2vec

if __name__ == '__main__':
    start_time = time.time()
    print('Training Word2Vec Model...')
    sentences = word2vec.LineSentence('./datasets/idwiki_new_lower.txt')
    id_w2v = word2vec.Word2Vec(sentences, vector_size=200, workers=multiprocessing.cpu_count()-1, sg=1, window=2)
    id_w2v.save('model/windows_2/idwiki_word2vec_200_skip-gram_window_2_new_lower.model')
    finish_time = time.time()

    print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))
    